In [ ]:
import cv2
import ipywidgets
from IPython.display import display
from ultralytics import YOLO
from jupyter_clickable_image_widget import ClickableImageWidget

# --- GStreamer CSI camera ---
width, height = 640, 480
pipeline = (
    f"nvarguscamerasrc ! "
    f"video/x-raw(memory:NVMM), width={width}, height={height}, format=NV12, framerate=30/1 ! "
    f"nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
)
camera = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
if not camera.isOpened():
    raise RuntimeError("Không mở được camera CSI")

# --- Load YOLO model ---
model = YOLO("/home/jetbot/flexin-boyz/model/best.pt")  # đường dẫn model YOLO của bạn

# --- Chuyển frame BGR -> JPEG bytes ---
def bgr8_to_jpeg(frame):
    _, jpeg = cv2.imencode('.jpg', frame)
    return jpeg.tobytes()

# --- Widget preview ---
camera_widget = ClickableImageWidget(width=width, height=height)
display(camera_widget)

# --- Camera loop với YOLO ---
from threading import Timer

def camera_loop():
    ret, frame = camera.read()
    if ret:
        # Chạy YOLO detection
        results = model(frame)[0]  # chỉ lấy result đầu tiên
        for box in results.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        camera_widget.value = bgr8_to_jpeg(frame)
    
    Timer(0.05, camera_loop).start()  # ~20 FPS

camera_loop()
